## Loading packages
Required packages for downstream analysis

In [1]:
inst <- suppressMessages(lapply(c('DESeq2',
                                  'vidger',
                                  'pheatmap',
                                  'ggplot2',
                                  'EnhancedVolcano',
                                  'VennDiagram',
                                  'org.Mm.eg.db',
                                  'clusterProfiler',
                                  'UpSetR',
                                  'readxl',
                                  'tidyverse'),
                                library,
                                character.only = TRUE)
) 

Set whether to produce plots, set to False for test runs.

In [2]:
bool_plot = FALSE

In [3]:
prot_input_file = '~/Documents/consultation/Luiza/omics_data/data/astrocyteprotoeme12102017-1.xlsx'
options(repr.plot.width=8, repr.plot.height=8)
my_palette <- colorRampPalette(c("blue", "white", "red"))(n=255)

In [4]:
dat <- read.csv(file="/Users/viktorian.miok/Documents/data/RNA-seq_Data-NEW/new_counts.csv", header=TRUE, sep=",")
rownames(dat) <- dat[,1]
dat <- dat[,-1]

In [5]:
group <- c(rep("HPT_Chow",6),rep("HPT_HFD",7),rep("Hippo_Chow",6),rep("Hippo_HFD",7),rep("CTX_Chow",6),rep("CTX_HFD",7))
lab <- colnames(dat)
loc <- c(rep("Hypothalamus",13), rep("Hippocampus",13), rep("Cortex",13))
diet <- c(rep("chow",6),rep("hfd",7),rep("chow",6),rep("hfd",7),rep("chow",6),rep("hfd",7))
id <- colnames(dat)
metaData <- data.frame(id, group, loc,diet)

In [6]:
# remove probleatic samples
dat <- dat[,-c(10,13,19,34,36)] #dat[,-c(10,13,19,36)] #10,13,
metaData <- metaData[-c(10,13,19,34,36),] #metaData[-c(10,13,19,36),] #10,13,

In [7]:
# convert from ensembl to symbol gene id
rownames(dat) <- gsub("\\..*","",rownames(dat))
sig.gene <- bitr(gsub("\\..*","",rownames(dat)), fromType="ENSEMBL",toType="SYMBOL",OrgDb=org.Mm.eg.db)

symb <- sig.gene$SYMBOL[ match(rownames(dat), sig.gene$ENSEMBL) ]
dat <- dat[!is.na(symb),]
symb <- symb[!is.na(symb)]
rownames(dat) <- make.names(symb,TRUE)

'select()' returned 1:many mapping between keys and columns

Warning message in bitr(gsub("\\..*", "", rownames(dat)), fromType = "ENSEMBL", :
“50.55% of input gene IDs are fail to map...”


In [8]:
# filtering
keep <- rowSums(dat) > 10
dat <- dat[keep,]

keep <- rowSums(dat >= 10) >= 5
dat <- dat[keep,]

## Comparison of diet effect for each tissue

In [9]:
dds <- DESeqDataSetFromMatrix(countData=dat, colData = metaData, design =~ group)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


In [10]:
dds <- DESeq(dds, fitType="parametric")

estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 148 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [11]:
#vst.data <- varianceStabilizingTransformation(dds, blind = TRUE, fitType = "parametric")

In [12]:
# variance stabilizing transformation data
vsd <- assay(vst(dds, 
                 fitType="local",
                 blind = TRUE
             )
        )

In [13]:
if(bool_plot){
    #dds <- estimateSizeFactors(dds)
    #idx <- rowMeans(counts(dds, normalized=TRUE)) >= 10
    #dds <- dds[idx,]
    #vsd <- vst(dds, fitType="local")
    pcaData <- plotPCA(vst(dds, fitType = "local"),
                       intgroup=c("group"),
                       returnData=TRUE
    )
    percentVar <- round(100 * attr(pcaData, "percentVar"))
    p2 <- ggplot(pcaData, aes(PC1, PC2, color = group)) +
                 geom_point(size=3)+
                 geom_label(aes(label = colnames(dat))) +
                 xlab(paste0("PC1: ",percentVar[1],"% variance")) + 
                 ylab(paste0("PC2: ",percentVar[2],"% variance")) +
                 ggtitle("PCA Plot") +
                 theme(legend.position = "right", legend.text = element_text(size = 10)) + 
                 guides(col = guide_legend(nrow=4))
    p2 + ylim(-30,35) + xlim(-80,80)
}

In [14]:
# Heatmpa of all samples and genes
if(bool_plot){  
    agsh <- pheatmap(vsd,
                     cluster_rows = T, 
                     scale = "row", 
                     show_rownames = FALSE, 
                     color = my_palette,
                     cluster_cols = T, 
                     annotation_col = as.data.frame(colData(dds)[,c("diet", "loc")]),
                     fontsize = 8,
                     breaks = seq(-5, 5, length.out = 255)
    )
    #save_pheatmap_pdf(agsh,
    #                  file.path(dir_fig, "all_genes_samples_heatmap.pdf"))
}

In [15]:
hpt <- results(dds, contrast=c("group", "HPT_Chow", "HPT_HFD"))
hippo <- results(dds, contrast=c("group", "Hippo_Chow", "Hippo_HFD"))
ctx <- results(dds, contrast=c("group", "CTX_Chow", "CTX_HFD"))

In [16]:
if(bool_plot){
    EnhancedVolcano(hpt,
                    lab = rownames(hpt),
                    pCutoff = 0.05,
                    FCcutoff = 10,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                    ylim = c(0, 8),
                    xlim = c(-8, 8),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "HPT Chow vs. HFD")
}

In [17]:
if(bool_plot){
    EnhancedVolcano(hippo,
                    lab = rownames(hippo),
                    pCutoff = 0.05,
                    FCcutoff = 10,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                    ylim = c(0, 8),
                    xlim = c(-8, 8),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "Hippo Chow vs. HFD")
}

In [18]:
if(bool_plot){
    EnhancedVolcano(ctx,
                    lab = rownames(ctx),
                    pCutoff = 0.05,
                    FCcutoff = 10,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                    ylim = c(0, 8),
                    xlim = c(-8, 8),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "CTX Chow vs. HFD")
}

In [19]:
if(bool_plot){
    df <- data.frame(location=c("Hypothalamus", "Hippocampus", "Cortex"),
                     Sig_genes=c(sum(hpt[,5]<0.05,na.rm=T), sum(hippo[,5]<0.05,na.rm=T), sum(ctx[,5]<0.05,na.rm=T)))
    ggplot(df, aes(x=location, y=Sig_genes, fill=location))  +
           geom_bar(stat="identity") + 
           theme_minimal() + 
           theme(legend.position="none") +
           scale_fill_manual(values=c("red", "#56B4E9", "olivedrab3"))
}

# Comparison between tissues

In [20]:
dds <- DESeqDataSetFromMatrix(countData=dat, colData = metaData, design =~ loc)
dds <- DESeq(dds)

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 215 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



In [21]:
hpt_vs_hippo <- results(dds, contrast=c("loc", "Hypothalamus", "Hippocampus"))
hpt_vs_ctx <- results(dds, contrast=c("loc", "Hypothalamus", "Cortex"))
ctx_vs_hippo <- results(dds, contrast=c("loc", "Cortex", "Hippocampus"))

In [22]:
if(bool_plot){
    EnhancedVolcano(hpt_vs_hippo,
                    lab = rownames(hpt_vs_hippo),
                    pCutoff = 0.05,
                    FCcutoff = 15,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                    ylim = c(0, 100),
                    xlim = c(-12, 12),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "HPT vs. Hippo")
}

In [23]:
if(bool_plot){
    EnhancedVolcano(hpt_vs_ctx,
                    lab = rownames(hpt_vs_ctx),
                    pCutoff = 0.05,
                    FCcutoff = 15,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                     ylim = c(0, 100),
                    xlim = c(-12, 12),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "HPT vs. CTX")
}

In [24]:
if(bool_plot){
    EnhancedVolcano(ctx_vs_hippo,
                    lab = rownames(ctx_vs_hippo),
                    pCutoff = 0.05,
                    FCcutoff = 15,
                    x = 'log2FoldChange',
                    y = 'pvalue',
                    ylim = c(0, 100),
                    xlim = c(-12, 12),
                    col = c("grey30", "grey30", "red2", "red2"),
                    title = "CTX vs. Hippo")
}

In [25]:
hpt_vs_hippo1 = hpt_vs_hippo[which(hpt_vs_hippo[,5]<0.05),]
hpt_vs_ctx1 = hpt_vs_ctx[which(hpt_vs_ctx[,5]<0.05),]
ctx_vs_hippo1 = ctx_vs_hippo[which(ctx_vs_hippo[,5]<0.05),]

In [26]:
if(bool_plot){
    df <- data.frame(location=c("HPT_vs_Hippo", "HPT_vs_CTX", "CTX_vs_Hippo"),
                     Sig_genes=c(sum(hpt_vs_hippo[,5]<0.05,na.rm=T), 
                                 sum(hpt_vs_ctx[,5]<0.05,na.rm=T), 
                                 sum(ctx_vs_hippo[,5]<0.05,na.rm=T))
    )
    ggplot(df, aes(x=location, y=Sig_genes, fill=location))  +
           geom_bar(stat="identity") +
           theme_minimal() +
           theme(legend.position = "none") +
           scale_fill_manual(values=c("purple", "brown", "blue"))
}

# Integrative Analysis Transcriptomics and Proteomics Data

In [27]:
# load the results from Persus
diffHippo <- read_excel(prot_input_file, sheet=10)[,c(4,6,7)]
colnames(diffHippo) <- c("GeneName","pvalue","log2FoldChange")
for (i in 1:nrow(diffHippo)) diffHippo[i,2] <- 10 ^ -diffHippo[i,2]

# load the results from Persus
diffCTX <- read_excel(prot_input_file, sheet=8)[,c(4,6,7)]
colnames(diffCTX) <- c("GeneName","pvalue","log2FoldChange")
for (i in 1:nrow(diffCTX)) diffCTX[i,2] <- 10 ^ -diffCTX[i,2]

# load the results from Persus
diffHPT <- read_excel(prot_input_file, sheet=6)[,c(4,6,7)]
colnames(diffHPT) <- c("GeneName","pvalue","log2FoldChange")
for (i in 1:nrow(diffHPT)) diffHPT[i,2] <- 10 ^ -diffHPT[i,2]

In [28]:
# cut-off
cutoff = 0.05

In [29]:
diffCTX <- diffCTX[which(diffCTX$pvalue<cutoff),]
diffHippo <- diffHippo[which(diffHippo$pvalue<cutoff),]
diffHPT <- diffHPT[which(diffHPT$pvalue<cutoff),]

In [30]:
hippo[,1] = rownames(hippo)
colnames(hippo) = c("GeneName","log2FoldChange","lfcSE","stat","pvalue","padj")
hippo <- as.data.frame(hippo)

hpt[,1] = rownames(hpt)
colnames(hpt) = c("GeneName","log2FoldChange","lfcSE","stat","pvalue","padj")
hpt <- as.data.frame(hpt)

ctx[,1] = rownames(ctx)
colnames(ctx) = c("GeneName","log2FoldChange","lfcSE","stat","pvalue","padj")
ctx <- as.data.frame(ctx)

In [31]:
ctx <- ctx[which(ctx$pvalue<cutoff),]
hippo <- hippo[which(hippo$pvalue<cutoff),]
hpt <- hpt[which(hpt$pvalue<cutoff),]

In [32]:
listInput <- list(CTX_transciptomics = rownames(ctx), 
                  Hippo_transciptomics = rownames(hippo),
                  HPT_transciptomics = rownames(hpt),
                  CTX_proteomics = as.matrix(diffCTX[,1]),
                  Hippo_proteomics = as.matrix(diffHippo[,1]),
                  HPT_proteomics = as.matrix(diffHPT[,1])
)

In [33]:
if(bool_plot){
    upset(fromList(listInput), 
          order.by = "freq", 
          text.scale = c(1.5, 1.5, 1.5, 1.5, 2, 1.5)
    )
}

In [34]:
vsd <- as.data.frame(assay(vst(dds, fitType="local", blind = TRUE)))

In [35]:
astro = c('Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9','Gfap','Aldh1l1','Gja1','Gjb6','Agt','Atp1b2')
neuro = c('Syp', 'Tubb3','Snap25','Syt1')
microglia = c('Itgam','Aif1')
oligodendrocyte = c('Mog','Mag')
endothelial = c('Slco1c1')
mural = c('Mustn1','Pdgfrb','Des')
tanycyes = c('Crym')
npc = c('Nes','Sox2','Prom1')
vlmc = c('Col1a1')

In [36]:
datfin <- data.frame(CTX=apply(vsd %>% select(starts_with("ctx_")), 1, mean),
                     HPT=apply(vsd %>% select(starts_with("hpt_")), 1, mean),
                     Hippo=apply(vsd %>% select(starts_with("hippo_")), 1, mean))

In [37]:
astro = datfin[rownames(datfin)%in%c('Slc1a2', 'Slc1a3', 'Aqp4', 'S100b', 'Sox9','Gfap','Aldh1l1','Gja1','Gjb6','Agt','Atp1b2'),]
neuro = datfin[rownames(datfin)%in%c('Syp', 'Tubb3','Snap25','Syt1'),]
mglia = datfin[rownames(datfin)%in%c('Itgam','Aif1'),]
oligo = datfin[rownames(datfin)%in%c('Mog','Mag'),]
endo = datfin[rownames(datfin)%in%c('Slco1c1'),]
mural = datfin[rownames(datfin)%in%c('Mustn1','Pdgfrb','Des'),]
tcyes = datfin[rownames(datfin)%in%c('Crym'),]
npc = datfin[rownames(datfin)%in%c('Nes','Sox2','Prom1'),]
vlmc = datfin[rownames(datfin)%in%c('Col1a1'),]

In [38]:
options(repr.plot.width=6, repr.plot.height=8)

In [39]:
if(bool_plot){
    mat <- as.matrix(rbind(astro,neuro,mglia,oligo,endo,mural,tcyes,npc,vlmc))
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255)
    
    Omics <- c(rep("transcriptomics",3))
    Acsa2 <- rep("Acsa2_pos",3)
    Tissue <- c("cortex","hypothalamus","hippocampus")
    dcol <- as.data.frame(cbind(Tissue,Acsa2,Omics))
    rownames(dcol) <- colnames(mat)

    Cell_types <- c(rep("astrocytes",11),rep("neurons",4),rep("microglia",2),rep("oligodendrocyte",2),
                    "endothelial",rep("mural",3),"tanycyes",rep("npc",3),"vlmc")
    drow <- as.data.frame(cbind(Cell_types))
    rownames(drow) <- rownames(mat)

    pheatmap(mat, scale = "column", show_rownames=T, color=my_palette, cluster_cols=F, cluster_rows=F, gaps_col=3,
             gaps_row=c(11,15,17,19,20,23,24,27,28), annotation_col=dcol, annotation_row=drow, fontsize=16,
             main="Acsa2 positive") 
}

# Heatmap clustering and enrichment

In [40]:
if(bool_plot){
    gen_heatmap =union(union(hpt[hpt$pvalue<0.05,1], ctx[ctx$pvalue<0.05,1]), hippo[hippo$pvalue<0.05,1])
    vsd_cl <- vsd[rownames(vsd)%in%gen_heatmap,]
    
    dim(vsd_cl)
    
    hr <- hclust(dist(vsd_cl, method = "maximum"), method="ward.D") 
    # define 5 row (gene) clusters using cutree outside ComplexHeatmap
    mycl.r <- cutree(hr, k=4)
    
    options(repr.plot.width=10, repr.plot.height=10)
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) # "blue","white","red" "dodgerblue","white","orangered"
    df <- as.data.frame(colData(dds)[,c("group", "loc")])
    annotation_row = data.frame(Cluster = factor(mycl.r))

    pheatmap(vsd_cl, cluster_rows=hr, scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=4, 
             cluster_cols=F, annotation_col=df, annotation_row=annotation_row, fontsize=8,main="All samples",
             breaks = seq(-4, 4, length.out = 255))
}

In [41]:
if(bool_plot){
    sig.gene <- bitr(names(mycl.r)[mycl.r==1], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
    #yy <- enrichGO(sig.gene[,2], 'org.Mm.eg.db', ont = "ALL", pvalueCutoff = 0.1)
    yy1 <- enrichKEGG(gene = sig.gene[,2], organism = 'mmu', pvalueCutoff = 0.1)
    barplot(yy1, showCategory=20)
}

In [42]:
if(bool_plot){
    sig.gene <- bitr(names(mycl.r)[mycl.r==2], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
    #yy <- enrichGO(sig.gene[,2], 'org.Mm.eg.db', ont = "ALL", pvalueCutoff = 0.1)
    yy2 <- enrichKEGG(gene = sig.gene[,2], organism = 'mmu', pvalueCutoff = 0.1)
    barplot(yy2, showCategory=20)
}

In [43]:
if(bool_plot){
    sig.gene <- bitr(names(mycl.r)[mycl.r==3], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
    #yy <- enrichGO(sig.gene[,2], 'org.Mm.eg.db', ont = "ALL", pvalueCutoff = 0.1)
    yy3 <- enrichKEGG(gene = sig.gene[,2], organism = 'mmu', pvalueCutoff = 0.1)
    barplot(yy3, showCategory=20)
}

In [44]:
if(bool_plot){
    sig.gene <- bitr(names(mycl.r)[mycl.r==4], fromType = "SYMBOL", toType = "ENTREZID", OrgDb = org.Mm.eg.db)
    #yy <- enrichGO(sig.gene[,2], 'org.Mm.eg.db', ont = "ALL", pvalueCutoff = 0.1)
    yy4 <- enrichKEGG(gene = sig.gene[,2], organism = 'mmu', pvalueCutoff = 0.1) 
    barplot(yy4, showCategory=20)
}

In [46]:
if(bool_plot){  
    clust1 = yy1@result[yy1@result[,6]<0.05,c(2,6,9)]
    clust1$Cluster = 'Cluster_1'
    clust2 = yy2@result[yy2@result[,6]<0.05,c(2,6,9)]
    clust2$Cluster = 'Cluster_2'
    clust3 = yy3@result[yy3@result[,6]<0.05,c(2,6,9)]
    clust3$Cluster = 'Cluster_3'
    clust4 = yy4@result[yy4@result[,6]<0.05,c(2,6,9)]
    clust4$Cluster = 'Cluster_4'

    options(repr.plot.width = 10, repr.plot.height = 15)
    ggplot(rbind(clust1, clust2, clust3, clust4), 
           aes(Cluster, Description, col = p.adjust, size = Count)) + 
           geom_point()+ 
           scale_colour_gradient(low = "red", high = "blue") +
           xlab("Clusters") + 
           ylab("KEGG enriched pathways") + 
          theme(text = element_text(size=20),axis.text.x = element_text(angle=45, hjust=1))
}

In [47]:
options(repr.plot.width = 10, repr.plot.height = 10)
# Heatmpa of all samples and genes
if(bool_plot){  
    agsh <- pheatmap(vsd,
                     cluster_rows = T, 
                     scale = "row", 
                     show_rownames = FALSE, 
                     color = my_palette,
                     cluster_cols = T, 
                     annotation_col = as.data.frame(colData(dds)[,c("diet", "loc")]),
                     fontsize = 8,
                     breaks = seq(-5, 5, length.out = 255)
    )
    #save_pheatmap_pdf(agsh,
    #                  file.path(dir_fig, "all_genes_samples_heatmap.pdf"))
}

In [48]:
options(repr.plot.width = 10, repr.plot.height = 10)
if(bool_plot){  
    agsh <- pheatmap(vsd,
                     cluster_rows = T, 
                     scale = "row", 
                     show_rownames = FALSE, 
                     color = my_palette,
                     cluster_cols = T, 
                     annotation_col = as.data.frame(colData(dds)[,c("diet", "loc")]),
                     fontsize = 8,
                     breaks = seq(-5, 5, length.out = 255)
    )
    #save_pheatmap_pdf(agsh,
    #                  file.path(dir_fig, "all_genes_samples_heatmap.pdf"))
}

### Per tissue heatmap

In [49]:
options(repr.plot.width = 10, repr.plot.height = 10)

In [50]:
if(bool_plot){
    vsd_cl <- vsd[rownames(vsd)%in%hpt[hpt$pvalue<0.01,1],]
    #hr <- hclust(dist(vsd_cl[,1:11], method = "euclidian"), method="complete") 
    # define 5 row (gene) clusters using cutree outside ComplexHeatmap
    #mycl.r <- cutree(hr, k=2)
    
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) # "blue","white","red" "dodgerblue","white","orangered"
    df <- as.data.frame(colData(dds)[1:11,c("group", "loc")])
    #annotation_row = data.frame(Cluster = factor(mycl.r))

    pheatmap(vsd_cl[,1:11], scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=2, 
             cluster_cols=F, annotation_col=df, fontsize=15,main="All samples",
             breaks = seq(-3, 3, length.out = 255))
}  #cluster_rows=hr, , cutree_rows=2   annotation_row=annotation_row,

In [51]:
if(bool_plot){
    vsd_cl <- vsd[rownames(vsd)%in%hippo[hippo$pvalue<0.01,1],]
    #hr <- hclust(dist(vsd_cl[,1:11], method = "euclidian"), method="complete") 
    # define 5 row (gene) clusters using cutree outside ComplexHeatmap
    #mycl.r <- cutree(hr, k=2)
    
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) # "blue","white","red" "dodgerblue","white","orangered"
    df <- as.data.frame(colData(dds)[12:23,c("group", "loc")])
    #annotation_row = data.frame(Cluster = factor(mycl.r))

    pheatmap(vsd_cl[,12:23], scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=2, 
             cluster_cols=F, annotation_col=df, fontsize=15,main="All samples",
             breaks = seq(-3, 3, length.out = 255))
}  #cluster_rows=hr, , cutree_rows=2   annotation_row=annotation_row,

In [52]:
if(bool_plot){
    vsd_cl <- vsd[rownames(vsd)%in%ctx[ctx$pvalue<0.01,1],]
    #hr <- hclust(dist(vsd_cl[,1:11], method = "euclidian"), method="complete") 
    # define 5 row (gene) clusters using cutree outside ComplexHeatmap
    #mycl.r <- cutree(hr, k=2)
    
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) # "blue","white","red" "dodgerblue","white","orangered"
    df <- as.data.frame(colData(dds)[24:34,c("group", "loc")])
    #annotation_row = data.frame(Cluster = factor(mycl.r))

    pheatmap(vsd_cl[,24:34], scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=2, 
             cluster_cols=F, annotation_col=df, fontsize=15,main="All samples",
             breaks = seq(-3, 3, length.out = 255))
}  #cluster_rows=hr, , cutree_rows=2   annotation_row=annotation_row,

In [53]:
if(bool_plot){
    hr <- hclust(dist(vsd_cl[,24:34], method = "maximum"), method="ward.D") 
    # define 5 row (gene) clusters using cutree outside ComplexHeatmap
    mycl.r <- cutree(hr, k=2)
    
    options(repr.plot.width=10, repr.plot.height=10)
    my_palette <- colorRampPalette(c("blue","white","red"))(n = 255) # "blue","white","red" "dodgerblue","white","orangered"
    df <- as.data.frame(colData(dds)[24:34,c("group", "loc")])
    annotation_row = data.frame(Cluster = factor(mycl.r))

    pheatmap(vsd_cl[,24:34], cluster_rows=hr, scale = "row", show_rownames=FALSE,color=my_palette, cutree_rows=2, 
             cluster_cols=F, annotation_col=df, annotation_row=annotation_row, fontsize=15,main="All samples",
             breaks = seq(-3, 3, length.out = 255))
}